In [1]:
import pandas as pd

# Load the CSV files into DataFrames
daily_sales = pd.read_csv('dailysales.csv')
weather_data = pd.read_csv('weatherdata.csv')
dept_sales = pd.read_csv('dptsales.csv')

# Function to display information about a DataFrame
def display_df_info(df, name):
    print(f"\n--- {name} ---")
    print("Columns:")
    print(df.columns.tolist())
    print("\nInfo:")
    df.info()
    print("\nDescription:")
    print(df.describe())
    print("\nFirst few rows:")
    print(df.head())

# Display information for each DataFrame
display_df_info(daily_sales, "Daily Sales")
display_df_info(weather_data, "Weather Data")
display_df_info(dept_sales, "Department Sales")


--- Daily Sales ---
Columns:
['Date', '# of Baskets', '# of Items', 'Avg Items', 'Sales $', 'Avg Sale', 'Scan Rate', 'Scan Sales']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321 entries, 0 to 1320
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          1321 non-null   object 
 1   # of Baskets  1321 non-null   int64  
 2   # of Items    1321 non-null   int64  
 3   Avg Items     1321 non-null   float64
 4   Sales $       1321 non-null   object 
 5   Avg Sale      1321 non-null   object 
 6   Scan Rate     1321 non-null   float64
 7   Scan Sales    1321 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 82.7+ KB

Description:
       # of Baskets   # of Items    Avg Items    Scan Rate
count   1321.000000  1321.000000  1321.000000  1321.000000
mean     132.939440   287.772142     2.162074    57.399546
std       32.362403    73.338931     0.220725    20.195469
min        0

In [5]:
import pandas as pd

# Load the CSV files
daily_sales = pd.read_csv('dailysales.csv')
weather_data = pd.read_csv('weatherdata.csv')
dept_sales = pd.read_csv('dptsales.csv')

# Function to display information about a DataFrame
def display_df_info(df, name):
    print(f"\n--- {name} ---")
    print("Columns:")
    print(df.columns.tolist())
    print("\nInfo:")
    df.info()
    print("\nDescription:")
    print(df.describe())
    print("\nFirst few rows:")
    print(df.head())

# Process dailysales.csv
def process_daily_sales(df):
    # Delete Scan Rate and Scan Sales
    df = df.drop(['Scan Rate', 'Scan Sales'], axis=1)
    
    # Rename columns
    df = df.rename(columns={
        '# of Baskets': 'totalbaskets',
        '# of Items': 'totalitems',
        'Sales $': 'totalsales',
        'Avg Items': 'avgitems',
        'Avg Sale': 'avgsale',
        'Date': 'date'
    })
    
    # Convert all headers to lowercase
    df.columns = [col.lower() for col in df.columns]
    
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Add day of week, day of month, and month
    df['dayofweek'] = df['date'].dt.day_name()
    df['dayofmonth'] = df['date'].dt.day
    df['month'] = df['date'].dt.month_name()
    
    # Reorder columns
    df = df[['dayofweek', 'dayofmonth', 'month', 'date', 'totalbaskets', 'totalitems', 'avgitems', 'totalsales', 'avgsale']]
    
    # Remove $ and commas from totalsales and avgsale and convert to float
    df['totalsales'] = df['totalsales'].str.replace('[\$,]', '', regex=True).astype(float)
    df['avgsale'] = df['avgsale'].str.replace('[\$,]', '', regex=True).astype(float)
    
    return df

# Process weatherdata.csv
def process_weather_data(df):
    # Keep specified columns
    keep_cols = ['datetime', 'tempmin', 'tempmax', 'temp', 'conditions', 'sunset', 'moonphase']
    df = df[keep_cols]
    
    # Rename datetime to date
    df = df.rename(columns={'datetime': 'date'})
    
    # Identify conditions
    conditions_list = ['rain', 'clear', 'overcast', 'partially cloudy', 'snow']
    
    # Create new columns for each condition
    for condition in conditions_list:
        df[condition] = df['conditions'].str.lower().str.contains(condition, na=False).map({True: 'yes', False: 'no'})
    
    # Delete conditions column
    df = df.drop('conditions', axis=1)
    
    # Rearrange columns in alphabetical order
    df = df[sorted(df.columns)]
    
    return df

# Process dptsales.csv
def process_dept_sales(df):
    # Delete Scan Rate and Scan Sales
    df = df.drop(['Scan Rate', 'Scan Sales'], axis=1)
    
    # Rename columns
    df = df.rename(columns={
        'Department': 'dpt',
        '# of Baskets': 'baskets',
        '# of Items': 'items',
        'Avg Items': 'avgitems',
        'Sales $': 'sales',
        'Avg Sale': 'avgsale'  # Fixed typo from 'Avg Sales'
    })
    
    # Remove $ and commas from sales and avgsale
    df['sales'] = df['sales'].str.replace('[\$,]', '', regex=True).astype(float)
    df['avgsale'] = df['avgsale'].str.replace('[\$,]', '', regex=True).astype(float)
    
    # Pivot the data
    pivot_df = df.pivot(index='Date', columns='dpt')
    
    # Flatten the multi-level column index
    pivot_df.columns = [f'{dpt}_{metric}' for metric, dpt in pivot_df.columns]
    
    # Reset index to make Date a column
    pivot_df = pivot_df.reset_index()
    
    # Sort columns to group department metrics together
    sorted_columns = ['Date'] + sorted([col for col in pivot_df.columns if col != 'Date'], key=lambda x: (x.split('_')[0], x))
    pivot_df = pivot_df[sorted_columns]
    
    return pivot_df

# Process and save the files
daily_sales_processed = process_daily_sales(daily_sales)
daily_sales_processed.to_csv('dly.csv', index=False)

weather_data_processed = process_weather_data(weather_data)
weather_data_processed.to_csv('wth.csv', index=False)

dept_sales_processed = process_dept_sales(dept_sales)
dept_sales_processed.to_csv('dpt.csv', index=False)

# Display information for the processed DataFrames
display_df_info(daily_sales_processed, "Daily Sales (dly.csv)")
display_df_info(weather_data_processed, "Weather Data (wth.csv)")
display_df_info(dept_sales_processed, "Department Sales (dpt.csv)")

<>:50: SyntaxWarning: invalid escape sequence '\$'
<>:51: SyntaxWarning: invalid escape sequence '\$'
<>:95: SyntaxWarning: invalid escape sequence '\$'
<>:96: SyntaxWarning: invalid escape sequence '\$'
<>:50: SyntaxWarning: invalid escape sequence '\$'
<>:51: SyntaxWarning: invalid escape sequence '\$'
<>:95: SyntaxWarning: invalid escape sequence '\$'
<>:96: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_25742/2683525160.py:50: SyntaxWarning: invalid escape sequence '\$'
  df['totalsales'] = df['totalsales'].str.replace('[\$,]', '', regex=True).astype(float)
/tmp/ipykernel_25742/2683525160.py:51: SyntaxWarning: invalid escape sequence '\$'
  df['avgsale'] = df['avgsale'].str.replace('[\$,]', '', regex=True).astype(float)
/tmp/ipykernel_25742/2683525160.py:95: SyntaxWarning: invalid escape sequence '\$'
  df['sales'] = df['sales'].str.replace('[\$,]', '', regex=True).astype(float)
/tmp/ipykernel_25742/2683525160.py:96: SyntaxWarning: invalid escape sequence '\$'
  df['avg


--- Daily Sales (dly.csv) ---
Columns:
['dayofweek', 'dayofmonth', 'month', 'date', 'totalbaskets', 'totalitems', 'avgitems', 'totalsales', 'avgsale']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321 entries, 0 to 1320
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dayofweek     1321 non-null   object        
 1   dayofmonth    1321 non-null   int32         
 2   month         1321 non-null   object        
 3   date          1321 non-null   datetime64[ns]
 4   totalbaskets  1321 non-null   int64         
 5   totalitems    1321 non-null   int64         
 6   avgitems      1321 non-null   float64       
 7   totalsales    1321 non-null   float64       
 8   avgsale       1321 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(2)
memory usage: 87.9+ KB

Description:
        dayofmonth                 date  totalbaskets   totalitems  \
count  13

In [6]:
import pandas as pd

# Load the CSV files
daily_sales = pd.read_csv('dly.csv')
weather_data = pd.read_csv('wth.csv')
dept_sales = pd.read_csv('dpt.csv')

# Function to display information about a DataFrame
def display_df_info(df, name):
    print(f"\n--- {name} ---")
    print("Columns:")
    print(df.columns.tolist())
    print("\nInfo:")
    df.info()
    print("\nFirst few rows:")
    print(df.head())
    print("\nMissing Values Check:")
    print(df.isnull().sum())

# Process dly.csv
def process_daily_sales(df):
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Encode dayofweek and month with numerical values
    day_map = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
    month_map = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 
                 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
    df['dayofweek'] = df['dayofweek'].map(day_map)
    df['month'] = df['month'].map(month_map)
    
    # Handle missing values for numerical columns
    numerical_cols = ['totalbaskets', 'totalitems', 'avgitems', 'totalsales', 'avgsale']
    for col in numerical_cols:
        for i in range(len(df)):
            if pd.isnull(df.at[i, col]):
                preceding = df[col].iloc[max(0, i-3):i].dropna()
                following = df[col].iloc[i+1:min(len(df), i+4)].dropna()
                if not preceding.empty or not following.empty:
                    avg_value = pd.concat([preceding, following]).mean()
                    df.at[i, col] = avg_value
    
    # Format numerical columns to two decimal places
    for col in numerical_cols:
        df[col] = df[col].round(2)
    
    return df

# Process wth.csv
def process_weather_data(df):
    # Encode binary columns
    binary_cols = ['overcast', 'partially cloudy', 'rain', 'snow']
    for col in binary_cols:
        df[col] = df[col].map({'yes': 1, 'no': 0})
    
    # Keep moonphase as is (already numerical)
    # Convert sunset to decimal time
    df['sunset'] = pd.to_datetime(df['sunset']).dt.strftime('%H:%M')
    df['sunset_decimal'] = df['sunset'].apply(lambda x: int(x.split(':')[0]) + int(x.split(':')[1])/60)
    df = df.drop('sunset', axis=1)
    
    # Format numerical columns to two decimal places
    numerical_cols = ['moonphase', 'temp', 'tempmax', 'tempmin', 'sunset_decimal']
    for col in numerical_cols:
        df[col] = df[col].round(2)
    
    return df

# Process dpt.csv
def process_dept_sales(df):
    # Replace all missing values with zero
    df = df.fillna(0)
    
    # Format numerical columns to two decimal places
    numerical_cols = df.select_dtypes(include=['float64']).columns
    for col in numerical_cols:
        df[col] = df[col].round(2)
    
    return df

# Process and save the files
daily_sales_processed = process_daily_sales(daily_sales)
daily_sales_processed.to_csv('dly.csv', index=False)

weather_data_processed = process_weather_data(weather_data)
weather_data_processed.to_csv('wth.csv', index=False)

dept_sales_processed = process_dept_sales(dept_sales)
dept_sales_processed.to_csv('dpt.csv', index=False)

# Display information for the processed DataFrames
display_df_info(daily_sales_processed, "Daily Sales (dly.csv)")
display_df_info(weather_data_processed, "Weather Data (wth.csv)")
display_df_info(dept_sales_processed, "Department Sales (dpt.csv)")


--- Daily Sales (dly.csv) ---
Columns:
['dayofweek', 'dayofmonth', 'month', 'date', 'totalbaskets', 'totalitems', 'avgitems', 'totalsales', 'avgsale']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321 entries, 0 to 1320
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dayofweek     1321 non-null   int64         
 1   dayofmonth    1321 non-null   int64         
 2   month         1321 non-null   int64         
 3   date          1321 non-null   datetime64[ns]
 4   totalbaskets  1321 non-null   int64         
 5   totalitems    1321 non-null   int64         
 6   avgitems      1321 non-null   float64       
 7   totalsales    1321 non-null   float64       
 8   avgsale       1321 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(5)
memory usage: 93.0 KB

First few rows:
   dayofweek  dayofmonth  month       date  totalbaskets  totalitems  \
0          4          19  

In [11]:
import pandas as pd

# Load the CSV files
daily_sales = pd.read_csv('dly.csv')
weather_data = pd.read_csv('wth.csv')
dept_sales = pd.read_csv('dpt.csv')

# Convert 'date' columns to datetime for consistency in merging
daily_sales['date'] = pd.to_datetime(daily_sales['date'])
weather_data['date'] = pd.to_datetime(weather_data['date'])
dept_sales['Date'] = pd.to_datetime(dept_sales['Date'])

# Rename 'Date' in dept_sales to 'date' for consistency
dept_sales = dept_sales.rename(columns={'Date': 'date'})

# Merge weather and daily sales on 'date'
combined_weather_sales = pd.merge(weather_data, daily_sales, on='date', how='inner')

# Merge the combined weather and sales with department sales on 'date'
final_combined_data = pd.merge(combined_weather_sales, dept_sales, on='date', how='inner')

# Reorder columns: date, dayofmonth, dayofweek, month, then the rest
columns_order = ['date', 'dayofmonth', 'dayofweek', 'month'] + \
                [col for col in final_combined_data.columns if col not in ['date', 'dayofmonth', 'dayofweek', 'month']]
final_combined_data = final_combined_data[columns_order]

# Save the final combined data to a new CSV file
final_combined_data.to_csv('cmbdata.csv', index=False)

# Function to display information about the combined DataFrame
def display_combined_info(df):
    print("\n--- Combined Data (cmbdata.csv) ---")
    print("Columns:")
    print(df.columns.tolist())
    print("\nInfo:")
    df.info()
    print("\nFirst few rows:")
    print(df.head())
    print("\nDescription of columns after 'month':")
    print(df.iloc[:, 4:].describe())

# Display the information
display_combined_info(final_combined_data)


--- Combined Data (cmbdata.csv) ---
Columns:
['date', 'dayofmonth', 'dayofweek', 'month', 'clear', 'moonphase', 'overcast', 'partially cloudy', 'rain', 'snow', 'temp', 'tempmax', 'tempmin', 'sunset_decimal', 'totalbaskets', 'totalitems', 'avgitems', 'totalsales', 'avgsale', 'Ashtrays_avgitems', 'Ashtrays_avgsale', 'Ashtrays_baskets', 'Ashtrays_items', 'Ashtrays_sales', 'Beer_avgitems', 'Beer_avgsale', 'Beer_baskets', 'Beer_items', 'Beer_sales', 'Candy_avgitems', 'Candy_avgsale', 'Candy_baskets', 'Candy_items', 'Candy_sales', 'Chips_avgitems', 'Chips_avgsale', 'Chips_baskets', 'Chips_items', 'Chips_sales', 'Cigarette_avgitems', 'Cigarette_avgsale', 'Cigarette_baskets', 'Cigarette_items', 'Cigarette_sales', 'Cigars_avgitems', 'Cigars_avgsale', 'Cigars_baskets', 'Cigars_items', 'Cigars_sales', 'D8 disposable_avgitems', 'D8 disposable_avgsale', 'D8 disposable_baskets', 'D8 disposable_items', 'D8 disposable_sales', 'D8 gummies_avgitems', 'D8 gummies_avgsale', 'D8 gummies_baskets', 'D8 gumm

In [15]:
import pandas as pd

# Load the CSV files
daily_sales = pd.read_csv('dly.csv')
weather_data = pd.read_csv('wth.csv')
dept_sales = pd.read_csv('dpt.csv')

# Process weather data to encode all weather condition columns as boolean
def process_weather_data(df):
    # Keep specified columns
    keep_cols = ['date', 'tempmin', 'tempmax', 'temp', 'clear', 'overcast', 'partially cloudy', 'rain', 'snow', 'moonphase', 'sunset_decimal']
    df = df[keep_cols]
    
    # Encode all weather condition columns as boolean
    weather_cols = ['clear', 'overcast', 'partially cloudy', 'rain', 'snow']
    for col in weather_cols:
        df[col] = df[col].map({'yes': True, 'no': False, 1: True, 0: False}).astype(bool)
    
    # Ensure numerical columns are rounded to two decimal places
    numerical_cols = ['moonphase', 'temp', 'tempmax', 'tempmin', 'sunset_decimal']
    for col in numerical_cols:
        df[col] = df[col].round(2)
    
    return df

# Process and save the updated weather data
weather_data_processed = process_weather_data(weather_data)
weather_data_processed.to_csv('wth.csv', index=False)

# Convert 'date' columns to datetime for consistency in merging
daily_sales['date'] = pd.to_datetime(daily_sales['date'])
weather_data_processed['date'] = pd.to_datetime(weather_data_processed['date'])
dept_sales['Date'] = pd.to_datetime(dept_sales['Date'])

# Rename 'Date' in dept_sales to 'date' for consistency
dept_sales = dept_sales.rename(columns={'Date': 'date'})

# Merge weather and daily sales on 'date'
combined_weather_sales = pd.merge(weather_data_processed, daily_sales, on='date', how='inner')

# Merge the combined weather and sales with department sales on 'date'
final_combined_data = pd.merge(combined_weather_sales, dept_sales, on='date', how='inner')

# Reorder columns: date, dayofmonth, dayofweek, month, then the rest
columns_order = ['date', 'dayofmonth', 'dayofweek', 'month'] + \
                [col for col in final_combined_data.columns if col not in ['date', 'dayofmonth', 'dayofweek', 'month']]
final_combined_data = final_combined_data[columns_order]

# Save the final combined data to a new CSV file
final_combined_data.to_csv('cmbdata.csv', index=False)

# Function to display information about the combined DataFrame
def display_combined_info(df):
    print("\n--- Combined Data (cmbdata.csv) ---")
    print("Columns:")
    print(df.columns.tolist())
    print("\nInfo:")
    df.info()
    print("\nFirst few rows:")
    print(df.head())
    print("\nDescription of columns after 'month':")
    print(df.iloc[:, 4:].describe())

# Display the information
display_combined_info(final_combined_data)


--- Combined Data (cmbdata.csv) ---
Columns:
['date', 'dayofmonth', 'dayofweek', 'month', 'tempmin', 'tempmax', 'temp', 'clear', 'overcast', 'partially cloudy', 'rain', 'snow', 'moonphase', 'sunset_decimal', 'totalbaskets', 'totalitems', 'avgitems', 'totalsales', 'avgsale', 'Ashtrays_avgitems', 'Ashtrays_avgsale', 'Ashtrays_baskets', 'Ashtrays_items', 'Ashtrays_sales', 'Beer_avgitems', 'Beer_avgsale', 'Beer_baskets', 'Beer_items', 'Beer_sales', 'Candy_avgitems', 'Candy_avgsale', 'Candy_baskets', 'Candy_items', 'Candy_sales', 'Chips_avgitems', 'Chips_avgsale', 'Chips_baskets', 'Chips_items', 'Chips_sales', 'Cigarette_avgitems', 'Cigarette_avgsale', 'Cigarette_baskets', 'Cigarette_items', 'Cigarette_sales', 'Cigars_avgitems', 'Cigars_avgsale', 'Cigars_baskets', 'Cigars_items', 'Cigars_sales', 'D8 disposable_avgitems', 'D8 disposable_avgsale', 'D8 disposable_baskets', 'D8 disposable_items', 'D8 disposable_sales', 'D8 gummies_avgitems', 'D8 gummies_avgsale', 'D8 gummies_baskets', 'D8 gumm